In [ ]:
import requests
import json
import re
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

默认方式:

In [ ]:
curl = input("请输入curl命令：")

需要先运行init.ipynb:

In [ ]:
# with open("tokens.txt", "r") as token_file:
#     curl=token_file.readline().strip()

In [ ]:
token_match = re.search(r'token:\s*([a-zA-Z0-9]+)', curl)
ic_cookie_match = re.search(r'ic-cookie=([a-zA-Z0-9-]+)', curl)
token = token_match.group(1)
ic_cookie = ic_cookie_match.group(1)


In [ ]:
datetime_now = datetime.now().strftime('%Y%m%d')
datetime_now=datetime_now.replace('28','29')

In [ ]:
url = 'http://libzwyy.jlu.edu.cn/ic-web/seatMenu'

headers = {
    'token': token,
}

cookies = {
    'ic-cookie': ic_cookie,
}

response = requests.get(url, headers=headers, cookies=cookies)
data_dict = json.loads(response.text)
learning_space_ids = []

def extract_ids(space, floor_name):
    if "children" in space:
        for child in space["children"]:
            learning_space_ids.append([floor_name + " - " + child["name"], child["id"]])

for floor in data_dict["data"]:
    extract_ids(floor, floor["name"])

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def on_change(selected_option):
    global option_value
    option_value = selected_option
    print(f"您选择的是: {selected_option}")

options = learning_space_ids

dropdown = widgets.interact(on_change, selected_option=options)

In [ ]:
url = 'http://libzwyy.jlu.edu.cn/ic-web/reserve?roomIds='+str(option_value)+'&resvDates='+datetime_now+'&sysKind=8'

response = requests.get(url, headers=headers, cookies=cookies)

if response.status_code == 200:
    data = json.loads(response.text)
    devices_data = data["data"]
else:
    print(f"Error: Request failed with status code {response.status_code}")

In [ ]:
# 提取 devName、startTime 和 endTime
table_data = []

for device in devices_data:
    dev_name = device["devName"]
    resv_info = device["resvInfo"]
    if len(resv_info) == 0:
        table_data.append([dev_name, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), datetime.now().strftime('%Y-%m-%d %H:%M:%S')])
    for resv in resv_info:
        start_time = datetime.fromtimestamp(resv["startTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        end_time = datetime.fromtimestamp(resv["endTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        table_data.append([dev_name, start_time, end_time])

In [ ]:
# 使用 pandas 创建 DataFrame
columns = ["devName", "startTime", "endTime"]
df = pd.DataFrame(table_data, columns=columns)

In [ ]:
df["startTime"] = pd.to_datetime(df["startTime"])
df["endTime"] = pd.to_datetime(df["endTime"])
df['duration'] = (df['endTime'] - df['startTime']).dt.seconds / 3600

In [ ]:
# 根据座位名称分组
groups = df.groupby("devName")

dev_names = df['devName'].unique()
name_to_index = {name: i for i, name in enumerate(dev_names)}

# 创建散点图
fig = go.Figure()

for name, group in groups:
    y_pos = name_to_index[name]
    for start_time, end_time in zip(group['startTime'], group['endTime']):
        fig.add_trace(
            go.Scatter(
                x=[start_time, end_time],
                y=[y_pos, y_pos],
                mode='lines',
                line=dict(color='orange', width=10),
                showlegend=False
            )
        )

# 设置x轴和y轴
fig.update_xaxes(title_text='时间', tickformat='%H:%M')
fig.update_yaxes(title_text='座位', tickvals=list(range(len(dev_names))), ticktext=dev_names)

# 设置布局
fig.update_layout(
    title='座位预约时间段分布',
    autosize=False,
    width=600,  # 调整图表宽度
    height=2400,  # 调整图表高度
    margin=dict(l=100, r=50, pad=4),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)

# 显示图表
fig.show()